In [266]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
%matplotlib
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from xgboost import XGBRFRegressor

Using matplotlib backend: Qt5Agg


# Train File

#### Import

In [267]:
df = pd.read_csv('Train.csv')

In [268]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   serial number     800 non-null    int64 
 1   account_info      800 non-null    object
 2   duration_month    800 non-null    int64 
 3   credit_history    800 non-null    object
 4   purpose           800 non-null    object
 5   credit_amount     800 non-null    int64 
 6   savings_account   800 non-null    object
 7   employment_st     800 non-null    object
 8   poi               800 non-null    int64 
 9   personal_status   800 non-null    object
 10  gurantors         800 non-null    object
 11  resident_since    800 non-null    int64 
 12  property_type     800 non-null    object
 13  age               800 non-null    int64 
 14  installment_type  800 non-null    object
 15  housing_type      800 non-null    object
 16  credits_no        800 non-null    int64 
 17  job_type        

In [269]:
df.describe()

,serial number,duration_month,credit_amount,poi,resident_since,age,credits_no,liables
count,800.0000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,400.5000,20.65125,3190.976250,2.966250,2.841250,35.406250,1.396250,1.147500
std,231.0844,12.15635,2732.671789,1.128806,1.106833,11.470317,0.569773,0.354825
min,1.0000,4.00000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,200.7500,12.00000,1342.250000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,400.5000,18.00000,2281.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,600.2500,24.00000,3914.250000,4.000000,4.000000,41.000000,2.000000,1.000000
max,800.0000,72.00000,15945.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [270]:
df.isna().sum()

serial number       0
account_info        0
duration_month      0
credit_history      0
purpose             0
credit_amount       0
savings_account     0
employment_st       0
poi                 0
personal_status     0
gurantors           0
resident_since      0
property_type       0
age                 0
installment_type    0
housing_type        0
credits_no          0
job_type            0
liables             0
telephone           0
foreigner           0
dtype: int64

#### y split

In [271]:
y = df['credit_amount']
df = df.drop((['serial number','credit_amount']),axis = 1)

#### label encoder

In [272]:
col = []
def convo(df):
    categorical_data = df.select_dtypes(exclude=[np.number])
    le = LabelEncoder()
    for i in categorical_data.columns:
        #print(dfex[i])
        xe = le.fit_transform(categorical_data[i])
        col.append(xe)
convo(df)
categorical_data = df.select_dtypes(exclude=[np.number])
non_categorical_data = df.select_dtypes(include=[np.number])
xat = pd.DataFrame(np.transpose(col), columns = categorical_data.columns)
final_data = pd.concat([xat,non_categorical_data], axis = 1)

In [273]:
final_data

,account_info,credit_history,purpose,savings_account,employment_st,personal_status,gurantors,property_type,installment_type,housing_type,job_type,telephone,foreigner,duration_month,poi,resident_since,age,credits_no,liables
0,0,4,4,4,4,2,0,0,2,1,2,1,0,6,4,4,67,2,1
1,1,2,4,0,2,1,0,0,2,1,2,0,0,48,2,2,22,1,1
2,3,4,7,0,3,2,0,0,2,1,1,0,0,12,2,3,49,1,2
3,0,2,3,0,3,2,2,1,2,2,2,0,0,42,2,4,45,1,2
4,0,3,0,0,2,2,0,3,2,2,2,0,0,24,3,4,53,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,3,2,3,1,1,1,0,1,2,0,2,0,0,9,2,4,22,1,1
796,0,2,1,4,4,2,0,1,2,2,2,1,0,18,1,4,51,1,2
797,3,4,3,0,1,1,0,1,2,0,1,0,0,12,2,4,22,2,1
798,3,3,0,4,4,3,0,2,2,1,2,1,0,24,4,4,54,2,1


#### removing unwanted variables using p_values

In [274]:
import statsmodels.api as sm

In [275]:
lr = sm.OLS(y,final_data)
model = lr.fit()

In [276]:
summary = model.summary2(50)
summary

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Ordinary least squares
================================================================================
Model:                  OLS              Adj. R-squared (uncentered): 0.810     
Dependent Variable:     50.0000          AIC:                         14308.4837
Date:                   2020-08-27 16:03 BIC:                         14397.4914
No. Observations:       800              Log-Likelihood:              -7135.2   
Df Model:               19               F-statistic:                 180.7     
Df Residuals:           781              Prob (F-statistic):          2.50e-270 
R-squared (uncentered): 0.815            Scale:                       3.3493e+06
---------------------------------------------------------------------------------
                        Coef.    Std.Err.     t      P>|t|     [0.025     0.975] 
---------------------------------------------------------------------------------
account_info           -51.0511   55.0418   -0.9275  0.3540  -159.0984    56.9962
credit_history         -31.7020   68.6990   -0.4615  0.6446  -166.5585   103.1545
purpose                -21.4117   23.7073   -0.9032  0.3667   -67.9492    25.1259
savings_account         80.7430   43.1755    1.8701  0.0618    -4.0107   165.4968
employment_st         -127.3473   58.8983   -2.1622  0.0309  -242.9650   -11.7296
personal_status        106.0459   92.1478    1.1508  0.2502   -74.8408   286.9326
gurantors               83.5012  139.4421    0.5988  0.5495  -190.2246   357.2269
property_type          280.1345   70.8029    3.9565  0.0001   141.1480   419.1210
installment_type        87.3767   87.6739    0.9966  0.3193   -84.7277   259.4810
housing_type            21.0024  140.8173    0.1491  0.8815  -255.4227   297.4275
job_type               604.0280  104.1044    5.8021  0.0000   399.6704   808.3856
telephone              650.1824  145.8814    4.4569  0.0000   363.8164   936.5485
foreigner              232.3619  370.9294    0.6264  0.5312  -495.7747   960.4985
duration_month         131.4994    5.6969   23.0827  0.0000   120.3164   142.6824
poi                   -745.4085   57.1438  -13.0444  0.0000  -857.5821  -633.2350
resident_since          17.8879   61.5146    0.2908  0.7713  -102.8657   138.6416
age                     11.9310    6.3989    1.8645  0.0626    -0.6302    24.4921
credits_no             243.8989  127.0005    1.9205  0.0552    -5.4038   493.2017
liables                134.4222  172.1328    0.7809  0.4351  -203.4756   472.3199
--------------------------------------------------------------------------------
Omnibus:                  293.995           Durbin-Watson:              2.042   
Prob(Omnibus):            0.000             Jarque-Bera (JB):           1494.969
Skew:                     1.602             Prob(JB):                   0.000   
Kurtosis:                 8.881             Condition No.:              248     
================================================================================

"""

thereshold is 0.5

credit_history,
gurantors,
housing_type,
foreigner,
resident_since

#### building X 

In [277]:
X = final_data.drop(['credit_history','gurantors','housing_type','foreigner','resident_since'], axis = 1)

In [278]:
X.head()

,account_info,purpose,savings_account,employment_st,personal_status,property_type,installment_type,job_type,telephone,duration_month,poi,age,credits_no,liables
0,0,4,4,4,2,0,2,2,1,6,4,67,2,1
1,1,4,0,2,1,0,2,2,0,48,2,22,1,1
2,3,7,0,3,2,0,2,1,0,12,2,49,1,2
3,0,3,0,3,2,1,2,2,0,42,2,45,1,2
4,0,0,0,2,2,3,2,2,0,24,3,53,2,2


In [279]:
X.columns

Index(['account_info', 'purpose', 'savings_account', 'employment_st',
       'personal_status', 'property_type', 'installment_type', 'job_type',
       'telephone', 'duration_month', 'poi', 'age', 'credits_no', 'liables'],
      dtype='object')

In [179]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 42)

#### Scaling

In [180]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [181]:
x_train[5]

array([-1.26924277, -0.84474924, -0.70316006,  1.34277678, -0.93006094,
        1.49912269, -0.94289634, -1.31912512, -0.83361577,  2.22281744,
        0.90190975,  1.93926339,  1.04264303, -0.38939887])

In [182]:
x_test[5]

array([ 1.13310948, -1.21579575, -0.70316006, -0.31326858,  0.48987943,
       -0.36700514, -2.34323744,  0.15750748, -0.83361577, -0.2515619 ,
        0.01040094, -0.30882377,  1.04264303, -0.38939887])

#### building functions for gridsearch, randomsearch, crossvalidation, and dataframe

In [183]:
def cross_sc(model):
    score = cross_val_score(model, x_train,y_train,cv = 10, scoring='r2')
    print('R2_Mean: ', np.mean(score))
    print('STD: ', np.std(score))

In [184]:
def rand_cv(model,X_train,y_train,param_grid):
    rand = RandomizedSearchCV(estimator=model,param_distributions=param_grid,scoring='r2',n_iter=20,verbose=1,cv=5, n_jobs=-1)
    rand_result = rand.fit(X_train, y_train)
    print('Best Score: ', rand_result.best_score_)
    print('Best Params: ', rand_result.best_params_)

In [185]:
def grid_cv(model,X_train,y_train,param_grid):
    grid = GridSearchCV(estimator=model,param_grid=param_grid,scoring='r2',verbose=1,cv=5, n_jobs=-1)
    grid_result = grid.fit(X_train, y_train)
    print('Best Score: ', grid_result.best_score_)
    print('Best Params: ', grid_result.best_params_)

In [186]:
def scr(Model,x_test = x_test):
    y2 = Model.predict(x_test)
    rmse = np.sqrt(np.mean((y_test-y2)**2))
    em = (1-rmse/100000)*100
    Model = Model
    RMSE = np.sqrt(metrics.mean_squared_error(y_test,y2))
    MSE = (metrics.mean_squared_error(y_test,y2))
    r2 = metrics.r2_score(y_test,y2)
    MAE = metrics.mean_absolute_error(y_test,y2)
    Dataset = pd.DataFrame(data = [[Model, RMSE, MSE, r2, MAE,em]], columns = ['Model','RMSE','MSE', 'r2', 'MAE','Evaluation_Metric'])
    return Dataset

In [187]:
def evaluation_metric(y_test, y_pred):
    rmse = np.sqrt(np.mean((y_test-y_pred)**2))
    em = (1-rmse/100000)*100
    return em

In [188]:
y.mean()

3190.97625

# x-x-x

# Models

In [189]:
lr = LinearRegression().fit(x_train,y_train)
lr_pred = lr.predict(x_test)

In [190]:
cross_sc(lr)

R2_Mean:  0.535663876460607
STD:  0.08192548156461821


In [191]:
scr(lr)

,Model,RMSE,MSE,r2,MAE,Evaluation_Metric
0,LinearRegression(),1888.35551,3.565887e+06,0.533249,1311.788359,98.111644


In [192]:
lr_pred[:10]

array([ 585.05732863, 6907.74014284, 7706.46852213, 2997.39348252,
       2427.76491336, 2508.08328521,  902.63239377, 2525.90987054,
       4815.35830212, 2476.81264022])

In [193]:
y_test[:10]

696     1103
667     3609
63     14421
533     1311
66      2171
621     1530
346      882
490     1934
760     5045
456     3905
Name: credit_amount, dtype: int64

### Lasso Regression

In [194]:
lasso = Lasso()
la = lasso.fit(x_train,y_train)
cross_sc(lasso)

R2_Mean:  0.535779548694439
STD:  0.08189187501988815


In [195]:
scr(lasso)

,Model,RMSE,MSE,r2,MAE,Evaluation_Metric
0,Lasso(),1888.277603,3.565592e+06,0.533288,1311.267629,98.111722


In [196]:
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)
param_grid

{'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

In [197]:
grid_cv(lasso,x_train,y_train,param_grid)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Score:  0.5335331263114018
Best Params:  {'alpha': 100}


[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    3.1s finished


In [198]:
rand_cv(lasso,x_train,y_train, param_grid)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best Score:  0.5335331263114018
Best Params:  {'alpha': 100}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    0.1s finished


In [233]:
lasso = Lasso(alpha=100)#0.1
la = lasso.fit(x_train,y_train)
cross_sc(lasso)

R2_Mean:  0.5326869028386578
STD:  0.07340005224589952


In [200]:
y_pred_lasso = lasso.predict(x_test)
Lassos = scr(lasso)
Lassos

,Model,RMSE,MSE,r2,MAE,Evaluation_Metric
0,Lasso(alpha=0.1),1888.347475,3.565856e+06,0.533253,1311.737216,98.111653


### Ridge Regression

In [201]:
ridge = Ridge()
ridge.fit(x_train,y_train)
cross_sc(ridge)

R2_Mean:  0.5357323977061378
STD:  0.08185008921791358


In [202]:
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)
param_grid

{'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

In [203]:
grid_cv(ridge,x_train,y_train,param_grid)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best Score:  0.5329107800970782
Best Params:  {'alpha': 10}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    0.1s finished


In [204]:
rand_cv(ridge,x_train,y_train,param_grid)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best Score:  0.5329107800970782
Best Params:  {'alpha': 10}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    0.0s finished


In [239]:
ridge = Ridge(alpha=10)#0.001
ridge.fit(x_train,y_train)
cross_sc(ridge)

R2_Mean:  0.5361819038033153
STD:  0.08118397302713273


In [240]:
Ridges = scr(ridge)
Ridges

,Model,RMSE,MSE,r2,MAE,Evaluation_Metric
0,Ridge(alpha=10),1890.14741,3.572657e+06,0.532363,1308.728083,98.109853


### Elestic Regression

In [241]:
elastic = ElasticNet()
el = elastic.fit(x_train,y_train)
cross_sc(el)

R2_Mean:  0.49332328062900216
STD:  0.06617417665504606


In [242]:
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
l1_ratio = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
param_grid = dict(alpha=alpha, l1_ratio = l1_ratio)
param_grid

{'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
 'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}

In [243]:
grid_cv(elastic,x_train,y_train,param_grid)

Fitting 5 folds for each of 70 candidates, totalling 350 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    0.4s


Best Score:  0.5335331263114018
Best Params:  {'alpha': 100, 'l1_ratio': 1}


[Parallel(n_jobs=-1)]: Done 350 out of 350 | elapsed:    0.7s finished


In [244]:
rand_cv(elastic,x_train,y_train,param_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Score:  0.5331221396260748
Best Params:  {'l1_ratio': 0.6, 'alpha': 0.1}


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished


In [246]:
elastic = ElasticNet(alpha=0.1, l1_ratio=0.6) #0.01 1
el = elastic.fit(x_train,y_train)
cross_sc(el)

R2_Mean:  0.5363596615927186
STD:  0.0803530170122


In [247]:
Elastic = scr(elastic)
Elastic

,Model,RMSE,MSE,r2,MAE,Evaluation_Metric
0,"ElasticNet(alpha=0.1, l1_ratio=0.6)",1893.24866,3.584390e+06,0.530827,1304.640308,98.106751


### Voting Regressor

In [248]:
from sklearn.ensemble import VotingRegressor

In [249]:
vot = VotingRegressor(estimators = [('Linear_regression',lr),('Lasso_regression',lasso),('Ridge_regression',ridge),('Elastic_regression',elastic)]).fit(x_train,y_train)

In [250]:
vot

VotingRegressor(estimators=[('Linear_regression', LinearRegression()),
                            ('Lasso_regression', Lasso(alpha=100)),
                            ('Ridge_regression', Ridge(alpha=10)),
                            ('Elastic_regression',
                             ElasticNet(alpha=0.1, l1_ratio=0.6))])

In [251]:
cross_sc(vot)

R2_Mean:  0.537623938185084
STD:  0.07917347206247834


In [252]:
scr(vot)

,Model,RMSE,MSE,r2,MAE,Evaluation_Metric
0,VotingRegressor(estimators=[('Linear_regressio...,1889.158564,3.568920e+06,0.532852,1294.601072,98.110841


## Decision Tree

In [130]:
dt = DecisionTreeRegressor(random_state=42)
cross_sc(dt)

R2_Mean:  0.22031447880913269
STD:  0.11991361349739961


In [131]:
param_grid = {'criterion':['mse','friedman_mse','mae'],
              'splitter':['best','random'],
              'max_depth':[1,2,3,4,5],
              'min_samples_split':[1,2,3,4,5],
              'min_samples_leaf':[1,2,3,4,5],
              'max_features':[1,2,3,4,5,6,7,8,9,10],
              'max_leaf_nodes':[1,2,3,4,5,6,7,8,9,10]}

In [132]:
rand_cv(dt,x_train,y_train,param_grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Score:  0.38100543988440966
Best Params:  {'splitter': 'best', 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_leaf_nodes': 10, 'max_features': 6, 'max_depth': 5, 'criterion': 'mse'}


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.4s finished


In [133]:
grid_cv(dt,x_train,y_train,param_grid)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 75000 candidates, totalling 375000 fits


[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 9720 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 25720 tasks      | elapsed:   34.7s


KeyboardInterrupt: 

In [134]:
dt = DecisionTreeRegressor(criterion='mse',splitter='best',max_depth=5,min_samples_split=2,min_samples_leaf=1,max_features=6,
                          max_leaf_nodes=10, random_state = 42)
dt.fit(x_train,y_train)

DecisionTreeRegressor(max_depth=5, max_features=6, max_leaf_nodes=10,
                      random_state=42)

In [135]:
cross_sc(dt)

R2_Mean:  0.3144662370155982
STD:  0.13778869138458094


In [136]:
scr(dt)

,Model,RMSE,MSE,r2,MAE,Evaluation_Metric
0,"DecisionTreeRegressor(max_depth=5, max_feature...",2234.584581,4.993368e+06,0.346401,1535.740615,97.765415


## Bagging

In [137]:
ada = AdaBoostRegressor()
cross_sc(ada)

R2_Mean:  0.44426709070145537
STD:  0.09402552773546717


In [138]:
param_grid = {'n_estimators':[100,500,1000,5000,10000,50000,100000],
              'learning_rate':[1,0.1,0.01,0.001,0.0001],
              'loss':['linear']}

In [ ]:
rand_cv(ada,x_train,y_train,param_grid)

In [142]:
ada = AdaBoostRegressor(n_estimators=500,loss='linear', learning_rate=1)
ada.fit(x_train,y_train)

AdaBoostRegressor(learning_rate=1, n_estimators=500)

In [143]:
cross_sc(ada)

R2_Mean:  0.45243416967944816
STD:  0.10325692591355182


In [144]:
scr(ada)

,Model,RMSE,MSE,r2,MAE,Evaluation_Metric
0,"(DecisionTreeRegressor(max_depth=3, random_sta...",2210.135415,4.884699e+06,0.360626,1660.764457,97.789865


In [145]:
ad_pre = ada.predict(x_test)

In [146]:
ad_pre[:10]

array([2374.41732283, 6507.5625    , 7664.94117647, 3264.57671958,
       2875.43283582, 2813.01219512, 2441.40522876, 2875.43283582,
       4623.025     , 2875.43283582])

In [147]:
y_test[:10]

696     1103
667     3609
63     14421
533     1311
66      2171
621     1530
346      882
490     1934
760     5045
456     3905
Name: credit_amount, dtype: int64

In [255]:
test = pd.read_csv('Test.csv')

In [256]:
test.head()

,serial number,account_info,duration_month,credit_history,purpose,savings_account,employment_st,poi,personal_status,gurantors,resident_since,property_type,age,installment_type,housing_type,credits_no,job_type,liables,telephone,foreigner
0,1,A14,24,A34,A46,A61,A75,4,A93,A101,4,A124,54,A143,A153,2,A173,2,A191,A201
1,2,A12,18,A34,A43,A61,A75,3,A92,A103,4,A121,48,A141,A151,2,A172,1,A192,A201
2,3,A11,20,A34,A42,A61,A75,1,A92,A101,4,A122,24,A143,A152,2,A173,1,A191,A201
3,4,A14,12,A34,A43,A65,A75,4,A93,A101,4,A123,35,A143,A152,2,A173,1,A191,A201
4,5,A12,12,A32,A40,A65,A71,1,A92,A101,2,A121,24,A143,A151,1,A171,1,A191,A201


In [257]:
test = test.drop(('serial number'),axis = 1)

In [258]:
test = test.drop(['credit_history','gurantors','housing_type','foreigner','resident_since'], axis = 1)

In [259]:
col = []
def convo(df):
    categorical_data = df.select_dtypes(exclude=[np.number])
    le = LabelEncoder()
    for i in categorical_data.columns:
        #print(dfex[i])
        xe = le.fit_transform(categorical_data[i])
        col.append(xe)
convo(test)
categorical_data = test.select_dtypes(exclude=[np.number])
non_categorical_data = test.select_dtypes(include=[np.number])
xat = pd.DataFrame(np.transpose(col), columns = categorical_data.columns)
final_data = pd.concat([xat,non_categorical_data], axis = 1)

In [260]:
test = final_data

In [261]:
sc = StandardScaler()
X = sc.fit_transform(X)
test = sc.transform(test)

In [262]:
vot.fit(X,y)

VotingRegressor(estimators=[('Linear_regression', LinearRegression()),
                            ('Lasso_regression', Lasso(alpha=100)),
                            ('Ridge_regression', Ridge(alpha=10)),
                            ('Elastic_regression',
                             ElasticNet(alpha=0.1, l1_ratio=0.6))])

In [263]:
scr(vot)

,Model,RMSE,MSE,r2,MAE,Evaluation_Metric
0,VotingRegressor(estimators=[('Linear_regressio...,1875.941484,3.519156e+06,0.539366,1281.755959,98.124059


In [264]:
vot_pred = vot.predict(test)

In [265]:
vot_pred[:10]

array([41994.68836269, 32466.18606898, 35681.73748812, 22221.91617694,
       23249.42909918, 60325.76351228, 12244.94257796, 24264.05985335,
       73697.43573714, 26808.85255011])

In [280]:
from joblib import dump
dump(vot, 'vot1.pkl') 

['vot1.pkl']